In [1]:
df <- read.csv("fifo.csv", header = TRUE, sep = ",")
#df$tally_q <- as.integer(df$tally_q)
# df$sale <- as.integer(df$sale)
# df$date <- as.Date(df$date, origin = "1899-12-30")
df$index <- seq(1:nrow(df))


head(df,3)
tail(df,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>
1,12/31/2022,Owned,1,0,AMC,1050,4,4273.5,1050,4273.5,0,1
2,1/27/2023,Sell,1,1,AMC,-1000,6,-5512.4,50,-5512.4,-1000,2
3,1/20/2023,Buy,2,1,AMC,1000,6,5625.0,1050,5625.0,0,3


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>
39,5/17/2023,Buy,23,16,AMC,1000,5,5009.0,2100,5009.0,0,39
40,5/30/2023,Sell,23,17,AMC,-1000,5,-4582.8,1100,-4582.8,-1000,40
41,45079.00,Sell,23,18,AMC,-1100,5,-4994.0,0,-4994.0,-1100,41


In [2]:
# create seperate buy / sell dfs

buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  # Filter sells
df_pl <- data.frame()

In [3]:
# create vars
buy_qty <- buys[1, 'quan']
sell_qty <- abs(sells[1, 'quan'])

buy_basis <- buys[1, 'basis']
sell_basis <- abs(sells[1, 'basis'])


# sell_quan_remainder <- buy_qty - abs(sell_qty)

cat('sell_qty : ', sell_qty, "\n")
cat('buy_qty : ', buy_qty, "\n")
cat('buy_basis :', buy_basis, "\n")
cat('sell_basis :', sell_basis, "\n")
# cat('remainder :',sell_quan_remainder , "\n")

# zero out vars
sells[1, 'tally_q'] <- 0
sells[1, 'tally_b'] <- 0
sells[1, 'Sale'] <- 0

sell_qty :  1000 
buy_qty :  1050 
buy_basis : 4273.5 
sell_basis : 5512.4 


In [4]:
head(buys, 3)
head(sells, 3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>
1,12/31/2022,Owned,1,0,AMC,1050,4,4273.5,1050,4273.5,0,1
3,1/20/2023,Buy,2,1,AMC,1000,6,5625.0,1050,5625.0,0,3
5,1/23/2023,Buy,3,2,AMC,1000,6,5788.0,1050,5788.0,0,5


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index,tally_q,Sale
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,6,-5512.40,50,0.00,-1000,2,0,0
4,2/3/2023,Sell,2,2,AMC,-1000,7,-6682.55,50,-6682.55,-1000,4,NA,NA
6,2/3/2023,Sell,3,3,AMC,-1000,6,-6255.00,50,-6255.00,-1000,6,NA,NA



## <font color = yellow> Keeper for concept above

In [5]:
df_sales <- data.frame()

# add index column

df$index <- seq(1, nrow(df))

buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  

### <font color= yellow> # Review for user prompt 

In [6]:
# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

# str(sell_quan)
# str(sell_price)


In [7]:
for(i in 1:4) {

repeat {
      
        if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("Index number is: ", df[i, "index"], "\n")
            sell_quan <- df[i, 'quan']
            sell_price <- df[i, 'pps']
            return(sell_quan)
        }
        i <- i + 1
     }
}

Got a Sell! 
Row number  2  Selling  1000 quantity of shares.  
Index number is:  2 


In [8]:


nrow(sells)

     if(buys[1, 'quan'] > sell_quan) {       
         
     # create New Balance
         buys[1,'quan'] <- buys[1,'quan'] - sell_quan
         buys[1,'basis'] <- buys[1, 'quan'] * buys[1, "pps"]
         buys[1,'tally_q'] <- sell_quan
         buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[1, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-1, ]
    } else if (buys[1, 'quan']  < sell_quan) {       
         cf1 <- buys[1,'quan']
         cf2 <- buys[1, "basis"]
         buys <- buys[-1, ]
         buys[1,'quan'] <- buys[1,'quan'] + cf1
         buys[1,'basis'] <- buys[1, 'basis'] + cf2
         
         buys[1,'quan'] <- buys[1,'quan'] - sell_quan
         buys[1,'basis'] <- buys[1, 'quan'] * buys[1, "pps"]
         buys[1,'tally_q'] <- sell_quan
         buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[1, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-1, ]
         }
    


nrow(sells)
head(buys, 3)     
head(df_sales)


[1] 18

[1] 17

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index,tally_q
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>
1,12/31/2022,Owned,1,0,AMC,2050,4,8200,1050,4273.5,-6000,1,-1000
3,1/20/2023,Buy,2,1,AMC,1000,6,5625,1050,5625.0,0,3,NA
5,1/23/2023,Buy,3,2,AMC,1000,6,5788,1050,5788.0,0,5,NA


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index,tally_q
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>
1,12/31/2022,Owned,1,0,AMC,2050,4,8200,1050,4273.5,-6000,1,-1000


In [9]:
head(sells,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>
4,2/3/2023,Sell,2,2,AMC,-1000,7,-6682.55,50,-6682.55,-1000,4
6,2/3/2023,Sell,3,3,AMC,-1000,6,-6255.00,50,-6255.00,-1000,6
8,2/3/2023,Sell,4,4,AMC,-1000,6,-6190.10,1050,-6190.10,-1000,8


In [10]:


# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

# nrow(sells)




#      if(buys[1, 'quan'] > sell_quan) {       
         
#      # create New Balance
#          buys[1,'quan'] <- buys[1,'quan'] - sell_quan
#          buys[1,'basis'] <- buys[1, 'quan'] * buys[1,"pps"]
#          buys[1,'tally_q'] <- sell_quan
#          buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
#          temp <- data.frame(buys[1, ])
#          df_sales <- rbind(df_sales, temp)
#          sells <- sells[-1, ]
#     } else if (buys[1, 'quan']  < sell_quan) {       
#          cf1 <- buys[1,'quan']
#          cf2 <- buys[1,"basis"]
#          buys <- buys[-1, ]
#          buys[1,'quan'] <- buys[1,'quan'] + cf1
#          buys[1,'basis'] <- buys[1, 'basis'] + cf2
         
#          buys[1,'quan'] <- buys[1,'quan'] - sell_quan
#          buys[1,'basis'] <- buys[1, 'quan'] * buys[1,"pps"]
#          buys[1,'tally_q'] <- sell_quan
#          buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
#          temp <- data.frame(buys[1, ])
#          df_sales <- rbind(df_sales, temp)
#          sells <- sells[-1, ]
#          }
    
    

# nrow(sells)
# head(buys, 3)     
# head(df_sales)


In [11]:
df <- read.csv("fifo.csv", header = TRUE, sep = ",")
df$index <- seq(1:nrow(df))

df_sales <- data.frame()
buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  

In [12]:
buy_ctr <- as.integer(1)
head(buys,3)
head(sells,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>
1,12/31/2022,Owned,1,0,AMC,1050,4,4273.5,1050,4273.5,0,1
3,1/20/2023,Buy,2,1,AMC,1000,6,5625.0,1050,5625.0,0,3
5,1/23/2023,Buy,3,2,AMC,1000,6,5788.0,1050,5788.0,0,5


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>
2,1/27/2023,Sell,1,1,AMC,-1000,6,-5512.40,50,-5512.40,-1000,2
4,2/3/2023,Sell,2,2,AMC,-1000,7,-6682.55,50,-6682.55,-1000,4
6,2/3/2023,Sell,3,3,AMC,-1000,6,-6255.00,50,-6255.00,-1000,6


## <font color = yellow> Change: don't delete rows, use the new ctr var in the df

In [13]:


# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)


for(i in 1:4) {
ct <- 0   
sell_quan <- as.integer(0)

cat("top of for loop, i ia: ", i, "\n") 
    
   

    repeat {
      
        if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("Index number is: ", df[i, "index"], "\n")
            sell_quan <- df[i, 'quan']
            return(sell_quan)
        }
        i <- i + 1
     }



  
    

     # Here there is no carryforward and the row counter can move forward
     # select row 

     if(buys[i, 'quan'] > sell_quan) {       
         
     cat("top of if # 1 ", "\n") 
      ct <- ct + buy_ctr   
         
    # create New Balance
         buys[i,'quan'] <- buys[i,'quan'] - sell_quan
         buys[i,'basis'] <- buys[i, 'quan'] * buys[i, "pps"]
         buys[i,'tally_q'] <- sell_quan
         buys[i,'sale'] <- buys[i,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[i, ])
         df_sales <<- rbind(df_sales, temp)
         i <<- i + 1
         sells <- sells[-1, ]
         # return
         
        # when the buys row i has quanity < sell quan, need carry values forward to next row,
         # And mo
       } else if (buys[i, 'quan']  < sell_quan) { 
         
          cat("top of else if # 1 ", "\n") 
         ct <- ct + i
         
         cfi <- buys[i,'quan']
         cf2 <- buys[i, "basis"]
         # delete row i
         buys <- buys[-i, ]
         # combine cf amounts to new row i
         buys[i,'quan'] <- buys[i,'quan'] + cfi
         buys[i,'basis'] <- buys[i, 'basis'] + cf2
         # resume as if the row's quan > sell quan  ( two vars in rowi updated)
         buys[i,'quan'] <- buys[i,'quan'] - sell_quan
         buys[i,'basis'] <- buys[i, 'quan'] * buys[i, "pps"]
         buys[i,'tally_q'] <- sell_quan
         buys[i,'sale'] <- buys[i,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[i, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-i, ]
         }
    
    }
    
print("--------")
cat("i value is: ", i, "\n")
print("--------")
print(ct)
print("--------")
nrow(sells)
print("--------")
head(buys, 3)
nrow(df_sales)
head(df_sales)


top of for loop, i ia:  1 
Got a Sell! 
Row number  2  Selling  1000 quantity of shares.  
Index number is:  2 
[1] "--------"
i value is:  2 
[1] "--------"
[1] 0
[1] "--------"


[1] 18

[1] "--------"


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tallq_q,tally_b,sale,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>
1,12/31/2022,Owned,1,0,AMC,1050,4,4273.5,1050,4273.5,0,1
3,1/20/2023,Buy,2,1,AMC,1000,6,5625.0,1050,5625.0,0,3
5,1/23/2023,Buy,3,2,AMC,1000,6,5788.0,1050,5788.0,0,5


[1] 0

<0 x 0 matrix>

In [14]:
stopper

ERROR: Error in eval(expr, envir, enclos): object 'stopper' not found


## <font color = yellow> Working code blocks below:


In [ ]:
for(i in 1:4) {
    df[which(df[i, "sell_ct"] == i & df[i, "type"] == "Sell"), ]
    
    cat("Row number ", i, "type is :", df[i, "type"], " The Sell qty: ", df$quan[i], " number of shares. ", "\n")
    
    }
    

In [ ]:
# df[which(df$B == value_to_find), ]

# df[which(df$A == value_to_find_A & df$B == value_to_find_B), ]


for(i in 1:4) {
    df[which(df$sell_ct == i & df$type == "Sell"), ]
    
    if(df[i,"type"] == "Sell") {
    cat("Row number ", i, " Selling ", df$quan[i], " number of shares. ", "\n")
    }
}


In [ ]:
ct <- 0 

for(i in 1:4) {

    repeat {
      
        if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("Index number is: ", df[i, "index"], "\n")
            return()
        }
        i <- i + 1
     }
}


  


In [ ]:
x <- 1
df_pl <- data.frame()
pl <- function(x) {
    
    sells[x, 'tally_b'] <<- buys[x, 'basis']
    
    sells[x, 'Sale'] <<- abs(sells[x, 'basis']) - sells[x, 'tally_b']
    temp <- data.frame(sells[x,])
    df_pl <<- rbind(df_pl, temp)
    
    }
 pl(x)
 head(df_pl, 3)
# head(sells, 3)
